In [14]:
# Import Files
#%pylab inline

from __future__ import print_function
import numpy as np
import os
import sys
import numpy as np
import pandas as pd
from IPython.display import display, Image
from scipy import ndimage
from six.moves import cPickle as pickle
from sklearn.metrics import accuracy_score
import tensorflow as tf

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [15]:
# Training Dataset Builder: Get the pixels, prepross them store the dataset using pickle files into the disk

image_size = 28

def image_pixel_standarize(image_pixel_val):
    return(image_pixel_val - 255.0/2)/255.0
    

def bld_pixels_arr(folder_name, min_no_of_image, max_no_of_image=None):
    # Build the pickle file for one Alphabet Image
    if max_no_of_image:
        image_filenames = os.listdir(folder_name)[0:max_no_of_image]
    else:
        image_filenames = os.listdir(folder_name)

    # Assign a numpy multudimensional array to store the image pixels, the image is 28*28 pixels
    dataset = np.ndarray(shape=(len(image_filenames), 
                                image_size, 
                                image_size
                               ),
                         dtype=np.float32)
    print (dataset.shape)
    for num_images, image in enumerate(image_filenames):
#         print (num_images)
        image_file_dir = os.path.join(folder_name, image)
        try:
            image_pixels = ndimage.imread(image_file_dir).astype(float)
            image_standarized = image_pixel_standarize(image_pixels)

            if image_standarized.shape != (image_size, image_size):
                raise Exception('Unexpected image shape: %s' % str(image_standarized.shape))
            dataset[num_images, :, :] = image_standarized
        except IOError as e:
            print('Could not read:', image, ':', e, '- hence skipping.')
        
    dataset = dataset[0:num_images+1, :, :]  # Will combine all the image pixels for 1 type Example All the 'a'
#     print (dataset.shape)
#     print (dataset)
    if num_images < min_no_of_image:
        raise Exception('Many fewer images than expected: %d < %d' %(num_images, min_no_of_image))

    print('Complete Training/Crossvalidation dataset :', dataset.shape)
    print('Mean:', np.mean(dataset))
    print('Standard deviation:', np.std(dataset))    
    return dataset


        
def crt_dataset(data_folders, dataset_type, min_no_of_image, max_no_of_image=None, force=None):    
    dataset_dir_n_names = []
    # One folder would comtain all the images pertaining to one particular alphabet.
    for folder in data_folders:  # [0:2]  
        print ('')
        print (folder)
        print ('Running for folder',os.path.basename(folder))
        set_filename = dataset_type + os.path.basename(folder) + '.pickle'
#         print (set_filename)
        dataset_dir_n_names.append(set_filename)
#         Ckech if the pickle file already exists, if not then we create one for each
        if os.path.exists(set_filename) and not force:
            print ('Yes+Yes--The pickle file exists')
        else:
            dataset = bld_pixels_arr(folder, min_no_of_image=min_no_of_image, max_no_of_image=max_no_of_image)
            try:
                with open(set_filename, 'wb') as f:
                    pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
            except Exception as e:
                print('Unable to save data to', set_filename, ':', e)
    return dataset_dir_n_names
                

DataPath_train = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataFolder/Training/"
data_folders_train = [os.path.join(DataPath_train,folders) for folders in os.listdir(DataPath_train) if folders!='.DS_Store']    
print (data_folders_train)

training_dataset_dir = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataPrepared/Training/'
# test_dataset_dir = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Udacity/Datapreparation/test_dataset/"

training_data_names_n_dir = crt_dataset(data_folders=data_folders_train, 
                            dataset_type=training_dataset_dir, 
                            min_no_of_image=4, 
                            max_no_of_image=None, 
                            force=None)
# test_data_names_n_dir = crt_dataset(data_folders=data_folders_test, 
#                             dataset_type=test_dataset_dir, 
#                             min_no_of_image=1800, 
#                             max_no_of_image=None, 
#                             force=None)

['/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataFolder/Training/a', '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataFolder/Training/b', '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataFolder/Training/c']

/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataFolder/Training/a
Running for folder a
Yes+Yes--The pickle file exists

/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataFolder/Training/b
Running for folder b
Yes+Yes--The pickle file exists

/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataFolder/Training/c
Running for folder c
Yes+Yes--The pickle file exists


In [16]:
# Create training and crossvalidation dataset 
def initialize_arrays(no_of_images, img_size):
    dataset = np.ndarray((no_of_images, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(no_of_images, dtype=np.int32)
    return dataset, labels

def train_valid_test_builder(training_data_names_n_dir, training_size, crossvalid_size=0):
    no_of_labels = len(training_data_names_n_dir)
    training_dataset, training_labels = initialize_arrays(training_size, image_size)
    if crossvalid_size!=0:
        crossvalid_dataset, crossvalid_labels = initialize_arrays(crossvalid_size, image_size)
    else:
        crossvalid_dataset, crossvalid_labels = None, None

    training_size_per_class = training_size // no_of_labels
    crossvalid_size_per_class = crossvalid_size // no_of_labels
    print ('training/teting size per class = ',training_size_per_class)
    print ('crossvalid size per class = ', crossvalid_size_per_class)
    
    start_v, start_t = 0, 0
    end_v, end_t = crossvalid_size_per_class, training_size_per_class
    end_vt = crossvalid_size_per_class+training_size_per_class

    for label, pickle_files in enumerate(training_data_names_n_dir):      
#         print ('Running for alphabet: ', os.path.basename(pickle_files))
        try:
            with open(pickle_files, 'rb') as f:
                letter_data = pickle.load(f)
                np.random.shuffle(letter_data)          # This will suffle the data which will give the top elements as random
                if crossvalid_dataset is not None:
                    crossvalid_for_letter = letter_data[0:crossvalid_size_per_class, :, :]
                    crossvalid_dataset[start_v:end_v, :, :] = crossvalid_for_letter
                    crossvalid_labels[start_v:end_v] = label
                    start_v += crossvalid_size_per_class
                    end_v += crossvalid_size_per_class

                training_for_letter = letter_data[crossvalid_size_per_class:end_vt, :, :]
                training_dataset[start_t:end_t, :, :] = training_for_letter
                training_labels[start_t:end_t] = label
                start_t += training_size_per_class
                end_t += training_size_per_class
        except Exception as e:
            print('Unable to process data from', pickle_files, ':', e)
            raise
    return crossvalid_dataset, crossvalid_labels, training_dataset, training_labels



# Load the pixel Data Stored
training_size = 27      # Total Train size including all different labels
crossvalid_size = 9     # Total Crossvalid size including all different labels
batch_size = 12

print (training_data_names_n_dir)
crossvalid_dataset, crossvalid_labels, training_dataset, training_labels = \
train_valid_test_builder(training_data_names_n_dir, training_size, crossvalid_size)



['/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataPrepared/Training/a.pickle', '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataPrepared/Training/b.pickle', '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataPrepared/Training/c.pickle']
training/teting size per class =  9
crossvalid size per class =  3


In [17]:
complete_dataset_path = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataPrepared/dataset_complete.p'

try:
    f = open(complete_dataset_path, 'wb')
    dataset_complete = {
        'training_dataset': training_dataset,
        'training_labels': training_labels,
        'crossvalid_dataset': crossvalid_dataset,
        'crossvalid_labels': crossvalid_labels,
#         'test_dataset': test_dataset,
#         'test_labels': test_labels,
    }
    pickle.dump(dataset_complete, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', complete_dataset_path, ':', e)
    raise

    
statinfo = os.stat(complete_dataset_path)
print('Compressed pickle size:', statinfo.st_size)

Compressed pickle size: 113426


In [18]:
# Loading The picke file and building mini-batches from the training dataset
complete_dataset_path = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataPrepared/dataset_complete.p'
with open(complete_dataset_path, 'rb') as f:
    fnl_dataset = pickle.load(f)
    training_dataset = (fnl_dataset['training_dataset'])
    training_labels = (fnl_dataset['training_labels'])
#     test_dataset = (fnl_dataset['test_dataset'])
#     test_labels = (fnl_dataset['test_labels'])
    crossvalid_dataset = (fnl_dataset['crossvalid_dataset'])
    crossvalid_labels = (fnl_dataset['crossvalid_labels'])
  


#Mini-Batch Creation
batch_indices = []
def batch_indice_creator(training_labels, batch_size_per_class):
    """Create batch with random samples and return appropriate format"""
    label_rand_arr = []
    for unq_labels in np.unique(training_labels):
        arr = np.array(np.where(training_labels==unq_labels)[0])
        np.random.shuffle(arr)
#         print ('the random array is: ', np.reshape(arr, (len(arr),1)))
        if np.any(label_rand_arr):
            label_rand_arr = np.hstack((label_rand_arr, np.reshape(arr, (len(arr),1))))
        else:
            label_rand_arr = np.reshape(arr, (len(arr),1))
    batch_indices = np.array_split(label_rand_arr.flatten(), batch_size_per_class)
    return batch_indices


training_batch_dir = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataPrepared/'        
batch_training_dict = {}
def batch_store(training_dataset, training_labels, batch_indices):
    for no, indices in enumerate(batch_indices):
        print (training_labels[indices])
        try:
            f = open(training_batch_dir+'batch'+str(no)+'.pickle', 'wb')
            batch = {
                'batch_train_dataset': training_dataset[indices],
                'batch_train_labels': training_labels[indices],
            }
            pickle.dump(batch, f, pickle.HIGHEST_PROTOCOL)
            f.close()
        except Exception as e:
            print('Unable to save data to', complete_dataset_path, ':', e)
            raise
            
batch_indices = batch_indice_creator(training_labels, 3)
batch_store(training_dataset, training_labels, batch_indices)



[0 1 2 0 1 2 0 1 2]
[0 1 2 0 1 2 0 1 2]
[0 1 2 0 1 2 0 1 2]


In [19]:
# Reshape the matrix and final matrix builder 
# tensor flow takes the labels input as binary code, where Alphabet A whose binary value is 0 will turn to a array
# with elements [1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0] and B becomes [0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
def reshape_data(dataset, labels, sample_size=None):
    no_of_labels = len(np.unique(labels))
    if sample_size:
        dataset = dataset[:sample_size].reshape(sample_size, image_size * image_size) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        labels = (np.arange(no_of_labels) == labels[:,None]).astype(np.float32)
    else:
        dataset = dataset.reshape(len(dataset), image_size * image_size) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        labels = (np.arange(no_of_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

# Retrieve the data of first batch:
training_batch_dir = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataPrepared/batch0.pickle'        
with open(training_batch_dir, 'rb') as f:
    batch_dataset = pickle.load(f)
    training_dataset_ = (batch_dataset['batch_train_dataset'])
    training_labels_ = (batch_dataset['batch_train_labels'])

training_dataset_, training_labels_ = reshape_data(training_dataset_, training_labels_)

print (training_labels_.shape)
print (training_dataset_.shape)

(9, 3)
(9, 784)


In [135]:
# # Declare and initialize the variables, In short Construct the whole Graph:
# no_inp_unit = 28*28
# no_hid_unit = 500
# no_output_unit = 3
# learning_rate = 0.01

# # Define Placeholders inputs and labels
# tf_training_dataset = tf.constant(training_dataset_)
# tf_training_labels = tf.constant(training_labels_)

###########  Practise  #############
import numpy as np
import os
import sys
import tensorflow as tf

no_inp_unit = 2
no_hid_unit = 3
no_output_unit = 2
no_of_labels = 2
momentum = 0.9
learning_rate = 0.1
batch_size = 2

training_dataset_ = np.array([[2,3],[4,5]], dtype='float32')
training_labels_ = (np.arange(no_of_labels) == np.array([0,1], dtype=float)[:,None]).astype(np.float32)
tf_training_dataset = tf.constant(training_dataset_)
tf_training_labels = tf.constant(training_labels_)
###########  Practise  #############

seed = 128
rng = np.random.RandomState(seed)

weights = {
    'input_to_hid_wghts': tf.Variable(tf.random_normal([no_inp_unit, no_hid_unit], seed=seed)),
    'hid_to_output_wghts': tf.Variable(tf.random_normal([no_hid_unit, no_output_unit], seed=seed)),
    'input_to_hid_wghts_delta': tf.Variable(tf.zeros([no_inp_unit, no_hid_unit])),
    'hid_to_output_wghts_delta': tf.Variable(tf.zeros([no_hid_unit, no_output_unit])),
}

biases = {
    'hid_bias' : tf.Variable(tf.zeros([no_hid_unit])),
    'output_bias' : tf.Variable(tf.zeros([no_output_unit])),
    'hid_bias_delta' : tf.Variable(tf.zeros([no_hid_unit])),
    'output_bias_delta' : tf.Variable(tf.zeros([no_output_unit]))
}


###### Forward Propagate ######
# Hidden Layer
input_to_hid_layer = tf.add(tf.matmul(tf_training_dataset, weights['input_to_hid_wghts']), biases['hid_bias'])
hid_layer_state = tf.sigmoid(inputs_to_hid, name=None)

# Output Layer
hid_to_output_layer = tf.add(tf.matmul(hid_layer_state, weights['hid_to_output_wghts']), biases['output_bias'])
output_layer_state = tf.nn.softmax(hid_to_output_layer, name=None)
error_derivative = tf.sub(output_layer_state, tf_training_labels, name=None)  # -(y - y_hat) = (y_hat - y)
# The above three lines of code can also be combined into one line as below.
loss_CE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hid_to_output_layer, tf_training_labels))


####### Back Propagate #######
# Output_Layer
hid_to_output_wghts_gradient =  tf.matmul(tf.transpose(hid_layer_state), error_derivative)
output_bias_gradient =  tf.reduce_sum(error_derivative, 0)  # sum all the row_value for each column 

# Hidden_Layer
back_propagate_deriv = tf.mul(tf.transpose(tf.matmul(weights['hid_to_output_wghts'], tf.transpose(error_derivative))),
                              tf.mul(hid_layer_state, 1.0-hid_layer_state)
                             )
input_to_hid_wghts_gradient = tf.matmul(tf.transpose(tf_training_dataset), back_propagate_deriv)
hid_bias_gradient = tf.reduce_sum(back_propagate_deriv, 0)

####### Weight Update  #######
# Update Weights
weights['input_to_hid_wghts_delta'] = tf.mul(momentum, weights['input_to_hid_wghts_delta']) \
                            + tf.div(input_to_hid_wghts_gradient, batch_size)
weights['input_to_hid_wghts'] = weights['input_to_hid_wghts'] - tf.mul(learning_rate,weights['input_to_hid_wghts_delta'])

weights['hid_to_output_wghts_delta'] = tf.mul(momentum, weights['hid_to_output_wghts_delta']) \
                            + tf.div(hid_to_output_wghts_gradient, batch_size)
weights['hid_to_output_wghts'] = weights['hid_to_output_wghts'] - tf.mul(learning_rate,weights['hid_to_output_wghts_delta'])


####### Bias Update  #######
biases['hid_bias_delta'] = tf.mul(momentum, biases['hid_bias_delta']) + tf.div(hid_bias_gradient, batch_size)
biases['hid_bias'] = biases['hid_bias'] - tf.mul(learning_rate,biases['hid_bias_delta'])

biases['output_bias_delta'] = tf.mul(momentum, biases['output_bias_delta']) + tf.div(output_bias_gradient, batch_size)
biases['output_bias'] = biases['output_bias'] - tf.mul(learning_rate,biases['output_bias_delta'])



#embed_to_hid_weights = embed_to_hid_weights - learning_rate * embed_to_hid_weights_delta

# Optimizer: Lets just use Gradient Descend
# learning_rate = 0.5
# optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_CE)

print (tf_training_dataset)
print (tf_training_labels)
print (weights)
print (biases)
print (inputs_to_hid)
print (hid_layer_state)
print (output_layer)

Tensor("Const_196:0", shape=(2, 2), dtype=float32)
Tensor("Const_197:0", shape=(2, 2), dtype=float32)
{'hid_to_output_wghts_delta': <tf.Tensor 'add_13:0' shape=(3, 2) dtype=float32>, 'input_to_hid_wghts_delta': <tf.Tensor 'add_12:0' shape=(2, 3) dtype=float32>, 'input_to_hid_wghts': <tf.Tensor 'sub_29:0' shape=(2, 3) dtype=float32>, 'hid_to_output_wghts': <tf.Tensor 'sub_30:0' shape=(3, 2) dtype=float32>}
{'hid_bias_delta': <tf.Tensor 'add_14:0' shape=(3,) dtype=float32>, 'output_bias': <tf.Tensor 'sub_32:0' shape=(2,) dtype=float32>, 'output_bias_delta': <tf.Tensor 'add_15:0' shape=(2,) dtype=float32>, 'hid_bias': <tf.Tensor 'sub_31:0' shape=(3,) dtype=float32>}
Tensor("Add_6:0", shape=(2, 3), dtype=float32)
Tensor("Sigmoid_61:0", shape=(2, 3), dtype=float32)
Tensor("Add_7:0", shape=(2, 2), dtype=float32)


In [144]:
# Now we create a session and run the session
init = tf.initialize_all_variables()

#print (init)

with tf.Session() as sess:
    sess.run(init)
    for i in range(2):
        print ('Forward Pass')
        print ('The input_to_hid_layer is: ', [w for no, w in enumerate(sess.run(input_to_hid_layer))])
        print ('')
        print ('The hid_layer_state is: ', [w for no, w in enumerate(sess.run(hid_layer_state))])
        print ('')
        print ('The hid_to_output_layer is: ', [w for no, w in enumerate(sess.run(hid_to_output_layer))])
        print ('')
        print ('The output_layer_state is: ', [w for no, w in enumerate(sess.run(output_layer_state))])
        print ('')
        print ('The error_derivative is: ', [w for no, w in enumerate(sess.run(error_derivative))])
        print ('')
        print ('The Loss value: ', sess.run(loss_CE))
        print ('')
        print ('')

        print ('Back Propagate')
        print ('The hid_to_output_wghts_gradient is: ', [w for no, w in enumerate(sess.run(hid_to_output_wghts_gradient))])
        print ('')
        print ('The hid_to_output_bias_gradient is: ', [w for no, w in enumerate(sess.run(hid_to_output_bias_gradient))])
        print ('')
        print ('The back_propagate_deriv is: ', [w for no, w in enumerate(sess.run(back_propagate_deriv))])
        print ('')
        print ('The input_to_hid_wghts_gradient is: ', [w for no, w in enumerate(sess.run(input_to_hid_wghts_gradient))])
        print ('')
        print ('')

        print ('Weight Update')
        print ('The input_to_hid_wghts_delta is: ', [w for no, w in enumerate(sess.run(weights['input_to_hid_wghts_delta']))])
        print ('')
        print ('The new input_to_hid_wghts is: ', [w for no, w in enumerate(sess.run(weights['input_to_hid_wghts']))])
        print ('')
        print ('The hid_to_output_wghts_delta is: ', [w for no, w in enumerate(sess.run(weights['hid_to_output_wghts_delta']))])
        print ('')
        print ('The new hid_to_output_wghts is: ', [w for no, w in enumerate(sess.run(weights['hid_to_output_wghts']))])
        print ('')
        print ('')

        print ('Bias Update')
        print ('The hid_bias_delta is: ', [w for no, w in enumerate(sess.run(biases['hid_bias_delta']))])
        print ('')
        print ('The new hid_bias is: ', [w for no, w in enumerate(sess.run(biases['hid_bias']))])
        print ('')
        print ('The output_bias_delta is: ', [w for no, w in enumerate(sess.run(biases['output_bias_delta']))])
        print ('')
        print ('The new output_bias is: ', [w for no, w in enumerate(sess.run(biases['output_bias']))])
        print ('')
        print ('')
        print ('')
        print ('')
        print ('')
        print ('')
    
    ### for each epoch, do:
    ###   for each batch, do:
    ###     create pre-processed batch
    ###     run optimizer by feeding batch
    ###     find cost and reiterate to minimize
    
#     for epoch in range(epochs):
#         avg_cost = 0
#         total_batch = int(train.shape[0]/batch_size)
#         for i in range(total_batch):
#             batch_x, batch_y = batch_creator(batch_size, train_x.shape[0], 'train')
#             _, c = sess.run([optimizer, cost], feed_dict = {x: batch_x, y: batch_y})
            
#             avg_cost += c / total_batch
            
#         print "Epoch:", (epoch+1), "cost =", "{:.5f}".format(avg_cost)
    
#     print "\nTraining complete!"

Forward Pass
The input_to_hid_layer is:  [array([ 0.02459085,  2.61364675, -3.66876554], dtype=float32), array([-0.46376991,  5.2827177 , -7.14768505], dtype=float32)]

The hid_layer_state is:  [array([ 0.50614744,  0.93173474,  0.02487347], dtype=float32), array([  3.86091858e-01,   9.94947135e-01,   7.86064717e-04], dtype=float32)]

The hid_to_output_layer is:  [array([-1.82842767,  1.17673755], dtype=float32), array([-1.83414948,  1.04686284], dtype=float32)]

The output_layer_state is:  [array([ 0.04719308,  0.95280689], dtype=float32), array([ 0.05310021,  0.94689977], dtype=float32)]

The error_derivative is:  [array([-0.95280695,  0.95280689], dtype=float32), array([ 0.05310021, -0.05310023], dtype=float32)]

The Loss value:  1.55404


Back Propagate
The hid_to_output_wghts_gradient is:  [array([-0.46175924,  0.46175921], dtype=float32), array([-0.83493143,  0.83493131], dtype=float32), array([-0.02365787,  0.02365787], dtype=float32)]

The hid_to_output_bias_gradient is:  [-0.8